In [26]:
import polars as pl
import numpy as np
import pandas as pd
import geopandas as gpd
from shapely import wkt


In [ ]:

for uid, tripdf in enumerate(pattern_df.groupby("unique_trip_vehicle_day")):
    print(uid)
    print(tripdf)
    break

In [63]:
#stuff to test
# pull out one pattern times
df = pd.read_parquet("cta-stop-watch/cta-stop-etl/out/parquets/2023-12-12.parquet")
df[df['rt'] == '6']['pid'].unique()
# grab one of the patterns for the 6 route 
# 14134
pattern_df = pd.read_parquet("cta-stop-watch/cta-stop-etl/out/pids/95.parquet")
# grab one trip to try stuff out
single_trip_df = pattern_df[pattern_df['unique_trip_vehicle_day'] == '7295.0235318404101004820292023-01-01']
single_trip_gdf = gpd.GeoDataFrame(
        single_trip_df,
        geometry=gpd.GeoSeries.from_xy(
            x=single_trip_df.loc[:, "lon"], y=single_trip_df.loc[:, "lat"], crs="EPSG:4326"
        ))
single_trip_gdf = single_trip_gdf[['unique_trip_vehicle_day','vid','data_time','geometry']]


In [74]:
segments_df = pd.read_parquet("cta-stop-watch/cta-stop-etl/out/patterns/pid_95_segment.parquet")
segments_df['geometry'] = segments_df['geometry'].apply(wkt.loads)
segments_gdf = gpd.GeoDataFrame(segments_df, crs='epsg:4326')
segments_gdf = segments_gdf[['segments','geometry']]

In [84]:
merged_df = segments_gdf.sjoin(single_trip_gdf, how="inner", predicate="contains")

In [89]:
merged_df

,segments,geometry,index_right,unique_trip_vehicle_day,vid,data_time,prev_segment
3,3,"POLYGON ((-87.63200 41.91079, -87.63206 41.910...",0,7295.0235318404101004820292023-01-01,2029,2023-01-01 05:57:00,2
4,4,"POLYGON ((-87.63206 41.91079, -87.63212 41.910...",0,7295.0235318404101004820292023-01-01,2029,2023-01-01 05:57:00,3
5,5,"POLYGON ((-87.63323 41.91078, -87.63329 41.910...",0,7295.0235318404101004820292023-01-01,2029,2023-01-01 05:57:00,4
30,30,"POLYGON ((-87.64548 41.91052, -87.64554 41.910...",1,7295.0235318404101004820292023-01-01,2029,2023-01-01 06:02:00,29
31,31,"POLYGON ((-87.64568 41.91052, -87.64574 41.910...",1,7295.0235318404101004820292023-01-01,2029,2023-01-01 06:02:00,30
...,...,...,...,...,...,...,...
434,434,"POLYGON ((-87.80335 41.90941, -87.80335 41.909...",15,7295.0235318404101004820292023-01-01,2029,2023-01-01 06:42:00,433
435,435,"POLYGON ((-87.80345 41.90912, -87.80348 41.909...",21,7295.0235318404101004820292023-01-01,2029,2023-01-01 06:52:00,434
435,435,"POLYGON ((-87.80345 41.90912, -87.80348 41.909...",18,7295.0235318404101004820292023-01-01,2029,2023-01-01 06:47:00,434
436,436,"POLYGON ((-87.80340 41.90910, -87.80341 41.909...",21,7295.0235318404101004820292023-01-01,2029,2023-01-01 06:52:00,435


In [87]:
merged_df["prev_segment"] = merged_df["segments"]-1
merged_df["seg_combined"] = merged_df["prev_segment"] + merged_df["segments"] / 2

In [91]:
merged_df

,segments,geometry,index_right,unique_trip_vehicle_day,vid,data_time,prev_segment,seg_combined
3,3,"POLYGON ((-87.63200 41.91079, -87.63206 41.910...",0,7295.0235318404101004820292023-01-01,2029,2023-01-01 05:57:00,2,3.5
4,4,"POLYGON ((-87.63206 41.91079, -87.63212 41.910...",0,7295.0235318404101004820292023-01-01,2029,2023-01-01 05:57:00,3,5.0
5,5,"POLYGON ((-87.63323 41.91078, -87.63329 41.910...",0,7295.0235318404101004820292023-01-01,2029,2023-01-01 05:57:00,4,6.5
30,30,"POLYGON ((-87.64548 41.91052, -87.64554 41.910...",1,7295.0235318404101004820292023-01-01,2029,2023-01-01 06:02:00,29,44.0
31,31,"POLYGON ((-87.64568 41.91052, -87.64574 41.910...",1,7295.0235318404101004820292023-01-01,2029,2023-01-01 06:02:00,30,45.5
...,...,...,...,...,...,...,...,...
434,434,"POLYGON ((-87.80335 41.90941, -87.80335 41.909...",15,7295.0235318404101004820292023-01-01,2029,2023-01-01 06:42:00,433,650.0
435,435,"POLYGON ((-87.80345 41.90912, -87.80348 41.909...",21,7295.0235318404101004820292023-01-01,2029,2023-01-01 06:52:00,434,651.5
435,435,"POLYGON ((-87.80345 41.90912, -87.80348 41.909...",18,7295.0235318404101004820292023-01-01,2029,2023-01-01 06:47:00,434,651.5
436,436,"POLYGON ((-87.80340 41.90910, -87.80341 41.909...",21,7295.0235318404101004820292023-01-01,2029,2023-01-01 06:52:00,435,653.0


In [ ]:
# row per ping with what segment it was in
# currently removes pings that are not in a segment
merged_df = segments_df.sjoin(single_trip_df, how="inner", predicate="within")

# create what segment it was between
merged_df["seq_combined"] = merged_df["prev_seq"] + merged_df["seq"] / 2

merged_df = merged_df[["seq_combined", "bus_timestamp", "bus_location"]].copy()

# add in the actual stops
# create stops df to add in
stops_df = segments_df[["seq", "geometry"]].copy()
stops_df.rename(
    {"seq": "seq_combined", "geometry": "stop_location"}, axis=1, inplace=True
)
stops_df["bus_timestamp"] = None
stops_df["bus_location"] = None

# concat the stops with the bus locations
final_df = pd.concat([merged_df, stops_df], axis=0)

final_df.sort_values(
    ["seq_combined", "bus_timestamp"], ascending=[True, True], inplace=True
)

return final_df